In [ ]:
!apt-get update
!apt-get install -y swig ffmpeg
!pip install gymnasium[box2d] stable-baselines3 shimmy moviepy numpy

In [ ]:
import gymnasium as gym
import numpy as np
import os
import glob
import base64
from pathlib import Path
from IPython.display import HTML, display

from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import torch.nn as nn

In [ ]:
# =========================
# 1. Mount Google Drive
# =========================
from google.colab import drive
drive.mount("/content/drive")

# =========================
# 2. Imports
# =========================
import gymnasium as gym
import numpy as np
import torch
import os

from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from gymnasium.wrappers import RecordVideo

# =========================
# 3. Environment Definition
# =========================
def make_eval_env(hardcore=False):
    env = gym.make(
        "BipedalWalker-v3",
        hardcore=hardcore,
        render_mode="rgb_array"
    )
    env = Monitor(env)
    env = RecordVideo(
        env,
        video_folder="/content/drive/MyDrive/bipedal_videos",
        episode_trigger=lambda ep: True,
        name_prefix="sac_eval_hardcore" if hardcore else "sac_eval_normal"
    )
    return env

# =========================
# 4. Load Environment + Normalization
# =========================
env = DummyVecEnv([lambda: make_eval_env(hardcore=False)])

env = VecNormalize.load(
    "/content/drive/MyDrive/bipedal_models/vecnorm_normal.pkl",
    env
)

# IMPORTANT: evaluation mode
env.training = False
env.norm_reward = False

# =========================
# 5. Load Model
# =========================
model = SAC.load(
    "/content/drive/MyDrive/bipedal_models/sac_normal",
    env=env
)

# =========================
# 6. Run Evaluation Episode
# =========================
obs = env.reset()
episode_reward = 0.0
step_count = 0

while True:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)

    episode_reward += reward[0]
    step_count += 1

    if done:
        break

env.close()

print("Episode reward:", episode_reward)
print("Episode length (steps):", step_count)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Episode reward: 318.02652
Episode length (steps): 730


In [ ]:
# =========================
# 1. Install & Imports
# =========================
# Ensure we have the video saver library
!pip install imageio[ffmpeg] > /dev/null 2>&1

import gymnasium as gym
import imageio
import numpy as np
import base64
from pathlib import Path
from IPython.display import HTML, display

from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# =========================
# 2. Setup (No Video Wrapper!)
# =========================
MODEL_PATH = "/content/drive/MyDrive/bipedal_models/sac_normal"
STATS_PATH = "/content/drive/MyDrive/bipedal_models/vecnorm_normal.pkl"
VIDEO_PATH = "manual_video.mp4"

def make_eval_env():
    # We DO NOT add RecordVideo here. We will render manually.
    return gym.make("BipedalWalker-v3", hardcore=False, render_mode="rgb_array")

# Create Env
env = DummyVecEnv([make_eval_env])
env = VecNormalize.load(STATS_PATH, env)
env.training = False    # Do not update stats
env.norm_reward = False # Do not normalize rewards

# Load Model
model = SAC.load(MODEL_PATH, env=env)

# =========================
# 3. Run & Capture Frames
# =========================
print("Simulating episode... capturing frames...")

frames = []
obs = env.reset()

while True:
    # 1. Get Action
    action, _ = model.predict(obs, deterministic=True)

    # 2. Step Environment
    obs, reward, done, info = env.step(action)

    # 3. Capture the Image MANUALLY
    # DummyVecEnv.render() returns the image of the first env
    frame = env.render()
    frames.append(frame)

    # 4. Stop if done
    if done[0]:
        print(f"Episode finished! Captured {len(frames)} frames.")
        break

env.close()

# =========================
# 4. Save & Display Video
# =========================
print("Saving video file...")
# Save the list of numpy arrays as an MP4
imageio.mimsave(VIDEO_PATH, frames, fps=30)

print(f"Video saved to {VIDEO_PATH}")

# Helper to display in Colab
def show_video(video_path):
    video_path = Path(video_path)
    video_b64 = base64.b64encode(video_path.read_bytes()).decode()
    display(HTML(f"""
    <div style="display:flex; justify-content:center;">
        <video width=640 height=480 controls autoplay>
            <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
        </video>
    </div>
    """))

show_video(VIDEO_PATH)

Simulating episode... capturing frames...


Episode finished! Captured 722 frames.
Saving video file...
Video saved to manual_video.mp4


In [ ]:
# =========================
# Imports
# =========================
import gymnasium as gym
import os, glob, base64
from pathlib import Path
from IPython.display import HTML, display

from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from gymnasium.wrappers import RecordVideo

# =========================
# Video Display Helper
# =========================
def show_video(path):
    path = Path(path)
    video = base64.b64encode(path.read_bytes()).decode()
    display(HTML(f"""
    <video width=640 height=480 controls autoplay>
        <source src="data:video/mp4;base64,{video}" type="video/mp4">
    </video>
    """))

# =========================
# Hardcore Environment
# =========================
def make_hardcore_eval_env():
    env = gym.make(
        "BipedalWalker-v3",
        hardcore=True,
        render_mode="rgb_array"
    )
    env = Monitor(env)
    env = RecordVideo(
        env,
        video_folder="/content/drive/MyDrive/bipedal_videos",
        name_prefix="sac_hardcore_eval",
        episode_trigger=lambda e: True,
        video_length=20000
    )
    return env

# =========================
# Build Environment
# =========================
env = DummyVecEnv([make_hardcore_eval_env])

env = VecNormalize.load(
    "/content/drive/MyDrive/bipedal_models/vecnorm_hardcore.pkl",
    env
)

env.training = False
env.norm_reward = False

# =========================
# Load Hardcore Model
# =========================
model = SAC.load(
    "/content/drive/MyDrive/bipedal_models/sac_hardcore",
    env=env
)

# =========================
# Run Forced Rollout
# =========================
obs = env.reset()

for _ in range(20000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)

env.close()

# =========================
# Show Video
# =========================
videos = glob.glob("/content/drive/MyDrive/bipedal_videos/*.mp4")
videos.sort(key=os.path.getmtime)

print("Showing:", videos[-1])
show_video(videos[-1])


Showing: /content/drive/MyDrive/bipedal_videos/sac_hardcore_eval-episode-16.mp4


In [ ]:
class LocomotionRewardWrapper(gym.Wrapper):
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)

        hull_angle = obs[0]
        forward_vel = obs[2]

        action_penalty = 0.001 * np.sum(action ** 2)
        joint_vel_penalty = 0.001 * np.sum(obs[6:14] ** 2)

        shaping = (
            1.5 * forward_vel
            - 0.8 * abs(hull_angle)
            - action_penalty
            - joint_vel_penalty
        )

        return obs, reward + shaping, terminated, truncated, info

In [ ]:
def make_env():
    env = gym.make("BipedalWalker-v3")
    env = LocomotionRewardWrapper(env)
    env = Monitor(env)
    return env

def make_hardcore_env():
    env = gym.make("BipedalWalker-v3", hardcore=True)
    env = LocomotionRewardWrapper(env)
    env = Monitor(env)
    return env

In [ ]:
N_ENVS = 1  # SAC is off-policy → single env is standard

env = DummyVecEnv([make_env])
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

policy_kwargs = dict(
   net_arch=[256, 256],
   activation_fn=nn.ReLU
)

model = SAC(
   "MlpPolicy",
   env,
   policy_kwargs=policy_kwargs,
   learning_rate=3e-4,
   buffer_size=1_000_000,
   batch_size=256,
   gamma=0.99,
   tau=0.005,
   ent_coef="auto",   # critical for SAC
   train_freq=1,
   gradient_steps=1,
   verbose=1
)

model.learn(total_timesteps=1_000_000)

os.makedirs("./models", exist_ok=True)
model.save("./models/sac_normal")
env.save("./models/vecnorm_normal.pkl")
env.close()


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os # Added this line

BASE_PATH = "/content/drive/MyDrive/bipedal_models"

os.makedirs(BASE_PATH, exist_ok=True)

model.save(f"{BASE_PATH}/sac_normal")
model.save_replay_buffer(f"{BASE_PATH}/sac_normal_replay.pkl")

env.save(f"{BASE_PATH}/vecnorm_normal.pkl")
env.close()

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import os

env = DummyVecEnv([make_hardcore_env])
env = VecNormalize(env, norm_obs = True, norm_reward = True, clip_obs = 10.0)
env.training = True
env.norm_reward = True

model = SAC.load(
    "/content/drive/MyDrive/bipedal_models/sac_normal",
    env=env,
    learning_rate=1e-4
)

model.learn(total_timesteps=1_000_000)

model.save("/content/drive/MyDrive/bipedal_models/sac_hardcore")
env.save("/content/drive/MyDrive/bipedal_models/vecnorm_hardcore.pkl")
env.close()


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os

BASE_PATH = "/content/drive/MyDrive/bipedal_models/hardcore"
os.makedirs(BASE_PATH, exist_ok=True)

model.save(f"{BASE_PATH}/sac_hardcore")
model.save_replay_buffer(f"{BASE_PATH}/sac_hardcore_replay.pkl")

env.save(f"{BASE_PATH}/vecnorm_hardcore.pkl")
env.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def evaluate(model_path, vecnorm_path, hardcore=False, n_eval=10):
    env = DummyVecEnv([lambda: gym.make("BipedalWalker-v3", hardcore=hardcore)])
    env = VecNormalize.load(vecnorm_path, env)
    env.training = False
    env.norm_reward = False

    model = SAC.load(model_path, env=env)

    returns, lengths = [], []

    for _ in range(n_eval):
        obs = env.reset()
        done = False
        ep_ret = 0
        ep_len = 0

        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _ = env.step(action)
            ep_ret += reward[0]
            ep_len += 1

        returns.append(ep_ret)
        lengths.append(ep_len)

    env.close()

    mean_r = np.mean(returns)
    mean_l = np.mean(lengths)

    solved = mean_r >= 300 and mean_l <= (2000 if hardcore else 1600)

    print("Hardcore:", hardcore)
    print("Mean Return:", mean_r)
    print("Mean Episode Length:", mean_l)
    print("Solved:", solved)

In [ ]:
def show_video(path):
    video = Path(path).read_bytes()
    b64 = base64.b64encode(video).decode()
    display(HTML(f"""
    <video width=640 height=480 controls>
        <source src="data:video/mp4;base64,{b64}" type="video/mp4">
    </video>
    """))

In [ ]:
from gymnasium.wrappers import RecordVideo
import shutil

def record_episode(hardcore=False, name="eval"):
    folder = f"./videos/{name}"
    shutil.rmtree(folder, ignore_errors=True)
    os.makedirs(folder, exist_ok=True)

    env = gym.make(
        "BipedalWalker-v3",
        hardcore=hardcore,
        render_mode="rgb_array"
    )

    env = RecordVideo(env, folder, episode_trigger=lambda x: True)

    env = DummyVecEnv([lambda: env])
    vec = "./models/vecnorm_hardcore.pkl" if hardcore else "./models/vecnorm_normal.pkl"
    model_p = "./models/sac_hardcore" if hardcore else "./models/sac_normal"

    env = VecNormalize.load(vec, env)
    env.training = False
    env.norm_reward = False

    model = SAC.load(model_p, env=env)

    obs = env.reset()
    for _ in range(2000):
        action, _ = model.predict(obs, deterministic=True)
        obs, _, done, _ = env.step(action)
        if done:
            break

    env.close()

    video = glob.glob(folder + "/*.mp4")[0]
    show_video(video)

In [ ]:
record_episode(hardcore=False, name="normal")

In [ ]:
record_episode(hardcore=True, name="hardcore")

In [ ]:
import gymnasium as gym
from stable_baselines3 import SAC # Changed from PPO to SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import imageio
import numpy as np
import os

# --- CONFIG ---
MODEL_PATH = "./models/sac_normal" # Corrected model path
STATS_PATH = "./models/vecnorm_normal.pkl"
OUTPUT_VIDEO = "manual_debug_run.mp4"

# 1. Setup Env (No RecordVideo Wrapper this time)
env = DummyVecEnv([lambda: gym.make("BipedalWalker-v3", render_mode="rgb_array")])

# 2. Load Stats & Model
print("Loading Brain & Eyes...")
if os.path.exists(STATS_PATH):
    env = VecNormalize.load(STATS_PATH, env)
    env.training = False
    env.norm_reward = False
else:
    print("⚠️ WARNING: No stats file found!")

model = SAC.load(MODEL_PATH, env=env) # Changed from PPO.load to SAC.load

# 3. RUN & CAPTURE FRAMES
print("--- 🎥 Starting Manual Recording ---")
obs = env.reset()
frames = []
steps = 0

for i in range(1600): # Try for 32 seconds max
    frames.append(env.render()) # Capture the frame manually

    action, _ = model.predict(obs, deterministic=True)
    obs, _, done, _ = env.step(action)
    steps += 1

    if done:
        print(f"❌ Robot fell/finished at Step {steps} ({steps/50:.2f} seconds)")
        break

env.close()

# 4. SAVE VIDEO
print(f"Saving {len(frames)} frames to video...")
if len(frames) > 0:
    # Save at 50 FPS (Real-time speed for BipedalWalker)
    imageio.mimsave(OUTPUT_VIDEO, frames, fps=50)
    print("✅ Video Saved!")

    # 5. DISPLAY
    from IPython.display import HTML, display
    import base64
    from pathlib import Path

    mp4 = Path(OUTPUT_VIDEO).read_bytes()
    b64 = base64.b64encode(mp4).decode()
    display(HTML(f'''
    <h3>Run Duration: {steps/50:.2f} Seconds</h3>
    <video width="640" controls autoplay>
      <source src="data:video/mp4;base64,{b64}" type="video/mp4">
    </video>
    '''))
else:
    print("No frames captured.")

In [ ]:
import gymnasium as gym
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import imageio
import numpy as np
import os
import base64
from pathlib import Path
from IPython.display import HTML, display

# --- CONFIG ---
MODEL_PATH = "./models/sac_hardcore"
STATS_PATH = "./models/vecnorm_hardcore.pkl"
OUTPUT_VIDEO = "manual_hardcore_run.mp4"

# 1. Setup Hardcore Env (hardcore=True)
# We use a simple lambda to create the gym env on the fly
env = DummyVecEnv([lambda: gym.make("BipedalWalker-v3", hardcore=True, render_mode="rgb_array")])

# 2. Load Stats & Model
print("Loading Hardcore Brain...")
if os.path.exists(STATS_PATH):
    env = VecNormalize.load(STATS_PATH, env)
    env.training = False
    env.norm_reward = False
else:
    print("⚠️ WARNING: No hardcore stats found! This will be ugly.")

if os.path.exists(f"{MODEL_PATH}.zip"):
    model = SAC.load(MODEL_PATH, env=env)
else:
    print("❌ ERROR: Hardcore model file missing.")
    # Stop here to prevent crashing later
    raise FileNotFoundError("Run the Hardcore training cell first.")

# 3. RUN & CAPTURE FRAMES
print("--- 🎥 Starting Manual Hardcore Recording ---")
obs = env.reset()
frames = []
steps = 0

# Hardcore tracks are long, we allow up to 2000 steps
for i in range(2000):
    frames.append(env.render()) # Capture the frame

    action, _ = model.predict(obs, deterministic=True)
    obs, _, done, _ = env.step(action)
    steps += 1

    if done:
        print(f"❌ Robot fell/finished at Step {steps} ({steps/50:.2f} seconds)")
        break

env.close()

# 4. SAVE & DISPLAY
if len(frames) > 0:
    print(f"Saving {len(frames)} frames...")
    imageio.mimsave(OUTPUT_VIDEO, frames, fps=50)

    mp4 = Path(OUTPUT_VIDEO).read_bytes()
    b64 = base64.b64encode(mp4).decode()
    display(HTML(f'''
    <h3>Hardcore Run: {steps/50:.2f} Seconds</h3>
    <video width="640" controls autoplay>
      <source src="data:video/mp4;base64,{b64}" type="video/mp4">
    </video>
    '''))
else:
    print("No frames captured.")